In [16]:
import torch
#files from drive
from google.colab import drive
drive.mount('/content/drive')

from torchvision import transforms, datasets
device = 'cuda' if torch.cuda.is_available() else 'cpu'


import pandas as pd
import numpy as np
from sklearn.utils import shuffle
seed = 42069
torch.backends.cudnn.deterministic = True
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

import os
from tqdm import tqdm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# create triplets id data frame
#for submission
test_triplet_file = 'drive/MyDrive/IML/test_triplets.txt'
test_triplet = pd.read_csv(test_triplet_file, sep=' ', names=['A','B','C'], header=None, dtype = str)

#train set
test_triplet_file = 'drive/MyDrive/IML/train_triplets.txt'
df = pd.read_csv(test_triplet_file, sep=' ', names=['A','B','C'], header=None, dtype = str)
df = shuffle(df,random_state=seed)

#split into training and validation triplets
train_size=int(len(df)*0.9)
train_triplet = df.iloc[:train_size]

# reverse half of the validation set
half_valid_size= int((len(df)-train_size)/2)
validation_triplet = df.iloc[train_size:]

valid_triplet_reverse = validation_triplet.iloc[half_valid_size:]
valid_triplet_reverse = pd.DataFrame({'A':valid_triplet_reverse['A'],'B':valid_triplet_reverse['C'],
                                      'C':valid_triplet_reverse['B'],'Label': np.zeros((half_valid_size,))})

valid_triplet_ordered = validation_triplet.iloc[:half_valid_size]
valid_triplet_ordered.insert(loc=3, column='Label', value=np.ones((valid_triplet_ordered.shape[0],)))

valid_triplet = shuffle(pd.concat([valid_triplet_reverse, valid_triplet_ordered]),random_state=seed)


## Apply pre-trained CNN to images

In [9]:

food_processed_dict = {}

import pickle
#read dict from file
save_file = "drive/MyDrive/IML/dict_food_preprocessed_50.pkl"
is_file_valid = False

#open if features dictionary alredy exist
if os.path.exists(save_file):
    file_dict = open(save_file,"rb")
    food_processed_dict = pickle.load(file_dict)
    file_dict.close()
    is_file_valid = len(food_processed_dict) == 1000

#extract features if they don't exist
if not os.path.exists(save_file) or not is_file_valid:

    import zipfile
    # Open .zip archive
    zip_folder = 'drive/MyDrive/food.zip'
    zip_ref = zipfile.ZipFile(zip_folder, 'r')
    zip_ref.extractall()
    zip_ref.close()

    from PIL import Image
    #use pretrained CNN to extract features of the images
    model_CNN = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
    model_CNN.to(device)
    model_CNN.fc = torch.nn.Identity()
    model_CNN.eval()

    reshape = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    def preprocess_image(image_path) :
      img = Image.open(image_path)
      input_tensor = reshape(img)
      input_batch = input_tensor.unsqueeze(0).to(device)
      with torch.no_grad():
          return model_CNN(input_batch).to('cpu')
    
    directory = 'food'
    
    # iterate over files 
    for filename in tqdm(os.listdir(directory)):
      img_path = os.path.join(directory, filename)
      # checking if it is a .jpg file
      if os.path.isfile(img_path) and (filename[-4:]==".jpg"):
        out = preprocess_image(img_path)
        food_processed_dict[filename[:-4]] = out

    #save dictionary to file
    file_dict = open(save_file,"wb")
    pickle.dump(food_processed_dict,file_dict)

    file_dict.close()

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

100%|██████████| 10001/10001 [04:36<00:00, 36.11it/s]


## Fully connected layers

In [10]:
def load_triplet_features(triplet,key):
    tensor = food_processed_dict[triplet[key]].flatten()
    return tensor.unsqueeze(0).to(device)

def load_triplet_to_torch(triplets):
      A_tensor_arr = [None] * len(triplets)
      B_tensor_arr = [None] * len(triplets)
      C_tensor_arr = [None] * len(triplets)

      if 'Label' in triplets.columns:
        labels_arr = [None] * len(triplets)

      for i in tqdm(range(len(triplets))):
          A_tensor_arr[i] = load_triplet_features(triplets.iloc[i],'A')
          B_tensor_arr[i] = load_triplet_features(triplets.iloc[i],'B')
          C_tensor_arr[i] = load_triplet_features(triplets.iloc[i],'C')
          if 'Label' in triplets.columns:
            labels_arr[i] = torch.full((1,),triplets['Label'].iloc[i]).unsqueeze(0)

      A_tensor_t = torch.cat(A_tensor_arr)
      B_tensor_t = torch.cat(B_tensor_arr)
      C_tensor_t = torch.cat(C_tensor_arr)
      if 'Label' in triplets.columns:
        label_t = torch.cat(labels_arr)
        return torch.utils.data.TensorDataset(A_tensor_t, B_tensor_t, C_tensor_t, label_t)

      else:
        return torch.utils.data.TensorDataset(A_tensor_t, B_tensor_t, C_tensor_t)

print("Loading verification triplets : ")
verif_ds = load_triplet_to_torch(valid_triplet)
verif_loader = torch.utils.data.DataLoader(verif_ds, batch_size=32, shuffle=False)

print("Loading train triplets : ")
train_ds = load_triplet_to_torch(train_triplet)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=False)

print("Loading test triplets : ")
test_ds = load_triplet_to_torch(test_triplet)

Loading verification triplets : 
Index(['A', 'B', 'C', 'Label'], dtype='object')


100%|██████████| 5952/5952 [00:05<00:00, 1060.69it/s]


Loading train triplets : 
Index(['A', 'B', 'C'], dtype='object')


100%|██████████| 53563/53563 [00:36<00:00, 1453.61it/s]


Loading test triplets : 
Index(['A', 'B', 'C'], dtype='object')


100%|██████████| 59544/59544 [00:37<00:00, 1592.28it/s]


In [18]:
class FullyConnectedLayers(torch.nn.Module):

  def __init__(self, input_dim: int):
    super().__init__() 

    self.fc1 = torch.nn.Linear(input_dim, 512) 
    self.fc2 = torch.nn.Linear(512, 82)
    self.fc3 = torch.nn.Linear(82,8)
    
    self.dropout = torch.nn.Dropout()
    self.activation_fn = torch.nn.ReLU()

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    
    result = self.dropout(x)
    result = self.fc1(result)
    result = self.activation_fn(result)
    result = self.fc2(result)
    result = self.activation_fn(result)
    result = self.fc3(result)
    
    return result
    

in_size = food_processed_dict[list(food_processed_dict.keys())[0]].shape[1]
model_fc = FullyConnectedLayers(in_size).to(device)

In [19]:
optim = torch.optim.Adam(model_fc.parameters(),lr=0.0001)
lp_norm = 1
loss_func = torch.nn.TripletMarginLoss(margin=1, p=lp_norm, eps=1e-06, swap=False, 
                                       size_average=None, reduce=None, reduction='mean')

In [20]:
#get label from the output of the network
def get_label(anchor, positive, negative):
  dist_p = torch.Tensor.norm(anchor-positive,p=lp_norm,dim=-1)
  dist_n = torch.Tensor.norm(anchor-negative,p=lp_norm,dim=-1)
  
  return dist_p < dist_n

def accuracy(anchor, positive, negative, labels) -> torch.Tensor:
  # computes the classification accuracy
  result = get_label(anchor, positive, negative)
  total_num = result.shape[0]
  correct_num = (result.flatten() == labels).sum().item()
  acc = correct_num / total_num
  
  assert 0. <= acc <= 1.
  return acc


def evaluate(model: torch.nn.Module) -> torch.Tensor:
  # goes through the test dataset and computes the test accuracy
  model.eval()  # bring the model into eval mode
  with torch.no_grad():
    acc_cum = 0.0
    num_eval_samples = 0
    for anchor_verif, positive_verif, negative_verif, labels in verif_loader:
      anc_emb = model(anchor_verif)      
      pos_emb = model(positive_verif)
      neg_emb = model(negative_verif)
      labels = labels.to(device)
      batch_size = pos_emb.shape[0]
      num_eval_samples += batch_size
    
      acc_cum += accuracy(anc_emb,pos_emb,neg_emb,labels.flatten()) * batch_size
      
    avg_acc = acc_cum / num_eval_samples
    assert 0 <= avg_acc <= 1
    return avg_acc

In [21]:

for epoch in range(31):
  # reset statistics trackers
  train_loss_cum = 0.0
  acc_cum = 0.0
  num_samples_epoch = 0
  
  for anchor, positive, negative in train_loader:
      # zero grads and put model into train mode
      optim.zero_grad()
      model_fc.train()
      # forward pass
      pos_emb = model_fc(positive)
      anc_emb = model_fc(anchor)
      neg_emb = model_fc(negative)
      loss = loss_func(anc_emb, pos_emb, neg_emb)
      # backward pass
      loss.backward()
      optim.step()

      # keep track of train stats
      num_samples_batch = positive.shape[0]
      num_samples_epoch += num_samples_batch
      train_loss_cum += loss / num_samples_batch
  
  # average the accumulated statistics
  avg_train_loss = train_loss_cum / num_samples_epoch
  valid_acc = evaluate(model_fc)
  
  # print some infos
  print(f'Epoch {epoch}, Train loss: {train_loss_cum:.4f}, Validation accuracy: {valid_acc:.4f}' )
  # save checkpoint of model
  if epoch % 5 == 0 and epoch > 0:
      save_path = f'model_epoch_{epoch}.pt'
      torch.save({'epoch': epoch,
                  'model_state_dict': model_fc.state_dict(),
                  'optimizer_state_dict': optim.state_dict()},
                 save_path)
      print(f'Saved model checkpoint to {save_path}')

Epoch 0, Train loss: 38.1206, Validation accuracy: 0.7144
Epoch 1, Train loss: 35.7228, Validation accuracy: 0.7320
Epoch 2, Train loss: 34.5390, Validation accuracy: 0.7382
Epoch 3, Train loss: 33.5637, Validation accuracy: 0.7463
Epoch 4, Train loss: 32.6379, Validation accuracy: 0.7520
Epoch 5, Train loss: 31.8779, Validation accuracy: 0.7586
Saved model checkpoint to model_epoch_5.pt
Epoch 6, Train loss: 31.2537, Validation accuracy: 0.7613
Epoch 7, Train loss: 30.3728, Validation accuracy: 0.7670
Epoch 8, Train loss: 29.8895, Validation accuracy: 0.7702
Epoch 9, Train loss: 29.1462, Validation accuracy: 0.7742
Epoch 10, Train loss: 28.7586, Validation accuracy: 0.7846
Saved model checkpoint to model_epoch_10.pt
Epoch 11, Train loss: 28.0260, Validation accuracy: 0.7868
Epoch 12, Train loss: 27.6269, Validation accuracy: 0.7891
Epoch 13, Train loss: 26.8853, Validation accuracy: 0.7969
Epoch 14, Train loss: 26.3903, Validation accuracy: 0.7962
Epoch 15, Train loss: 25.7514, Validat

In [22]:
def reload_model(number):
    previous_run = torch.load(f'./model_epoch_{number}.pt')
    previous_run['model_state_dict']
    model_fc.load_state_dict(previous_run['model_state_dict'])
    optim.load_state_dict(previous_run['optimizer_state_dict'])
reload_model(30)

In [24]:
model_fc.eval()  # bring the model into eval mode
test_loader = torch.utils.data.DataLoader(test_ds,batch_size=59544,shuffle=False)
with torch.no_grad():
  for anchor, positive, negative in test_loader:
    pos_emb = model_fc(positive)
    anc_emb = model_fc(anchor)
    neg_emb = model_fc(negative)
  
    submission = get_label(anc_emb,pos_emb,neg_emb)
answer = submission.flatten().to('cpu').numpy().astype(int)
#write to file 
pd.DataFrame(answer).transpose().to_csv("submission.txt",index=False,header=False,sep='\n')